In [ ]:
!pip install caer canaro

In [ ]:
import os
import caer
import canaro
import numpy as np
import gc
import cv2 as cv

In [ ]:
img_size = (80,80)  # images should be of same size
channels = 1       # 1-> grayscale image
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
# we will go through all the files and store the no. of images in each file in a dictionary and sort 
# it in decreasing order and grab the top 10 elements of the dictionary

In [ ]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
    
# sorting dictionary in descending order
char_dict = caer.sort_dict(char_dict,descending = True)
char_dict

In [ ]:
# grabbing first 10 names from the dictionary and storing it in a list

In [ ]:
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count+=1
    if count>=10:
        break
characters

In [ ]:
# creating the training data

In [ ]:
train = caer.preprocess_from_dir(char_path,characters,channels=channels,IMG_SIZE=img_size,isShuffle=True)

In [ ]:
len(train)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0],cmap='gray')
plt.show()

In [ ]:
# separating features and labels

In [ ]:
featureSet , label = caer.sep_train(train,IMG_SIZE=img_size)

In [ ]:
# normalizing the featureSet in range of (0,1)

In [ ]:
from tensorflow.keras.utils import to_categorical
featureSet = caer.normalize(featureSet)
label = to_categorical(label,len(characters))

In [ ]:
# creating training and validation data

In [ ]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, label, val_ratio=0.2)
x_train = np.array(x_train)
y_train = np.array(y_train)
x_val = np.array(x_val)
y_val = np.array(y_val)

In [ ]:
# deleting variables that are no longer of use

In [ ]:
del train
del featureSet
del label
gc.collect()

In [ ]:
# image data generator
# this is basically an image generator that will synthesize new images from already existing images to
# help introduce some randomness in our network and make it perform better

In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
datagen = canaro.generators.imageDataGenerator()
train_gen = datagen.flow(x_train,y_train, batch_size=BATCH_SIZE)

In [ ]:
# creating the model

In [ ]:
model = canaro.models.createSimpsonsModel(IMG_SIZE=img_size,channels=channels,output_dim=len(characters),
                                         loss='binary_crossentropy',decay=1e-6,learning_rate=0.001,
                                         momentum=0.9,nesterov=True)

In [ ]:
model.summary()

In [ ]:
# creating callbacks list
# callbacks list will contain something called learning rate schedule that will essentially schedule
# the learning rate at specific intervals so that our network can essentially train better

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [ ]:
# training the model

In [ ]:
training = model.fit(train_gen,steps_per_epoch=len(x_train)//BATCH_SIZE,epochs=EPOCHS,
                     validation_data=(x_val,y_val),validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks = callbacks_list)

In [ ]:
characters

In [ ]:
test_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset/lisa_simpson/pic_0000.jpg'
img = cv.imread(test_path)
plt.imshow(img,cmap='gray')
plt.show()

In [ ]:
def prepare(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.resize(image, img_size)
    image = caer.reshape(image, img_size, 1)
    return image

In [ ]:
predictions = model.predict(prepare(img))


In [ ]:
print(characters[np.argmax(predictions[0])])